In [1]:
%matplotlib inline
import numpy as np

import quantify_core.visualization.pyqt_plotmon as pqm
import quantify_core.data.handling as dh
from quantify_core.analysis import base_analysis as ba
from quantify_core.analysis import cosine_analysis as ca
from quantify_core.data.handling import (
    default_datadir,
    set_datadir,
)
from quantify_core.measurement import MeasurementControl
from quantify_core.utilities.examples_support import mk_cosine_instrument
from quantify_core.utilities.experiment_helpers import create_plotmon_from_historical

from quantify_core.utilities.inspect_utils import display_source_code
from quantify_core.visualization.instrument_monitor import InstrumentMonitor

In [2]:
set_datadir(default_datadir())

Data will be saved in:
C:\Users\z5440260\quantify-data


In [3]:
import qcodes as qc
qc.logger.start_all_logging()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\z5440260\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


KeyboardInterrupt: 

In [ ]:
meas_ctrl = MeasurementControl("meas_ctrl")
plotmon = pqm.PlotMonitor_pyqt("plotmon")
meas_ctrl.instr_plotmon(plotmon.name)
#insmon = InstrumentMonitor("InstrumentMonitor")

In [ ]:
pars = mk_cosine_instrument()

In [ ]:
pars.acq_delay(1)
pars.noise_level(101)

In [ ]:
meas_ctrl.settables(
    [pars.t, pars.freq]
)  # as a QCoDeS parameter, 't' obeys the JSON schema for a valid Settable and can be passed to the meas_ctrl directly.
meas_ctrl.setpoints_grid([
    np.linspace(0, 2, 22), 
    np.linspace(0, 2, 22)
])
meas_ctrl.gettables(
    pars.sig
)  # as a QCoDeS parameter, 'sig' obeys the JSON schema for a valid Gettable and can be passed to the meas_ctrl directly.
# opx_plus_driver.prepare_quantify_measurement(meas_ctrl)

pars.t.batched = True
pars.freq.batched = True
pars.sig.batched = True
pars.sig.batch_size = 22*11

In [ ]:
pars.sig.get_raw

In [ ]:
dset = meas_ctrl.run("my experiment", soft_avg=100)

In [ ]:
from arbok.samples.sunshine.readout.other_ST_read import OtherStReadout